In [ ]:
import hassapi as hass
from datetime import datetime, timedelta

class DashboardIntegration:
    def __init__(self, controller):
        self.controller = controller
        self.hass = controller.hass

    def _create_dashboard_entities(self):
        """Create custom sensors and input helpers for dashboard integration"""

        # Create sensors for each room
        for room in self.controller.rooms:
            room_name = room.replace('.', '_')

            # Temperature comparison sensor
            self.hass.set_state(
                f"sensor.climate_{room_name}_temp_status",
                self._get_temp_status(room),
                attributes={
                    "friendly_name": f"{room} Temperature Status",
                    "icon": "mdi:thermometer",
                    "current_temp": self.controller.get_current_temp(room),
                    "target_temp": self.controller.get_target_temp(room),
                    "unit_of_measurement": "°C"
                }
            )

            # Climate action sensor
            self.hass.set_state(
                f"sensor.climate_{room_name}_action",
                self.controller.get_climate_action(room),
                attributes={
                    "friendly_name": f"{room} Climate Action",
                    "icon": self._get_action_icon(self.controller.get_climate_action(room))
                }
            )

            # Eco mode status sensor
            self.hass.set_state(
                f"binary_sensor.climate_{room_name}_eco_mode",
                self.controller.is_eco_mode_active(room),
                attributes={
                    "friendly_name": f"{room} Eco Mode",
                    "device_class": "power",
                    "icon": "mdi:leaf" if self.controller.is_eco_mode_active(room) else "mdi:leaf-off"
                }
            )

        # System-wide sensors
        self.hass.set_state(
            "sensor.climate_season_detection",
            self.controller.get_current_season(),
            attributes={
                "friendly_name": "Weather-based Season",
                "icon": self._get_season_icon(self.controller.get_current_season()),
                "outdoor_temp": self.controller.get_outdoor_temp(),
                "detection_method": "weather_based"
            }
        )

        self.hass.set_state(
            "sensor.climate_power_source",
            self.controller.get_active_power_source(),
            attributes={
                "friendly_name": "Active Power Source",
                "icon": self._get_power_icon(self.controller.get_active_power_source()),
                "solar_available": self.controller.is_solar_available(),
                "grid_cost": self.controller.get_current_grid_cost()
            }
        )

    def _create_input_helpers(self):
        """Create input helpers for easy dashboard control"""

        # Eco temperature thresholds
        self.hass.call_service("input_number/create", {
            "object_id": "climate_eco_heating_threshold",
            "name": "Eco Heating Threshold",
            "min": 15,
            "max": 25,
            "step": 0.5,
            "unit_of_measurement": "°C",
            "icon": "mdi:thermometer-low"
        })

        self.hass.call_service("input_number/create", {
            "object_id": "climate_eco_cooling_threshold",
            "name": "Eco Cooling Threshold",
            "min": 20,
            "max": 30,
            "step": 0.5,
            "unit_of_measurement": "°C",
            "icon": "mdi:thermometer-high"
        })

        # System enable/disable
        self.hass.call_service("input_boolean/create", {
            "object_id": "climate_system_enabled",
            "name": "Smart Climate System",
            "icon": "mdi:home-thermometer"
        })

        self.hass.call_service("input_boolean/create", {
            "object_id": "climate_eco_mode_enabled",
            "name": "Eco Mode Enabled",
            "icon": "mdi:leaf"
        })

        # Room-specific overrides
        for room in self.controller.rooms:
            room_name = room.replace('.', '_')

            # Temperature override
            self.hass.call_service("input_number/create", {
                "object_id": f"climate_{room_name}_temp_override",
                "name": f"{room} Temperature Override",
                "min": 15,
                "max": 30,
                "step": 0.5,
                "unit_of_measurement": "°C",
                "icon": "mdi:thermometer-lines"
            })

            # Mode selection
            self.hass.call_service("input_select/create", {
                "object_id": f"climate_{room_name}_mode",
                "name": f"{room} Climate Mode",
                "options": ["auto", "heat", "cool", "off", "eco"],
                "icon": "mdi:hvac"
            })

            # Schedule override
            self.hass.call_service("input_boolean/create", {
                "object_id": f"climate_{room_name}_schedule_override",
                "name": f"{room} Schedule Override",
                "icon": "mdi:calendar-clock"
            })

    def _update_dashboard_sensors(self):
        """Update dashboard sensors with current values"""

        for room in self.controller.rooms:
            room_name = room.replace('.', '_')

            # Update temperature status
            current_temp = self.controller.get_current_temp(room)
            target_temp = self.controller.get_target_temp(room)
            temp_diff = current_temp - target_temp

            self.hass.set_state(
                f"sensor.climate_{room_name}_temp_status",
                f"{temp_diff:+.1f}°C",
                attributes={
                    "friendly_name": f"{room} Temperature Status",
                    "icon": "mdi:thermometer",
                    "current_temp": current_temp,
                    "target_temp": target_temp,
                    "difference": temp_diff,
                    "unit_of_measurement": "°C",
                    "status": self._get_temp_status_text(temp_diff)
                }
            )

            # Update climate action
            action = self.controller.get_climate_action(room)
            self.hass.set_state(
                f"sensor.climate_{room_name}_action",
                action,
                attributes={
                    "friendly_name": f"{room} Climate Action",
                    "icon": self._get_action_icon(action),
                    "last_changed": datetime.now().isoformat()
                }
            )

            # Update eco mode status
            eco_active = self.controller.is_eco_mode_active(room)
            self.hass.set_state(
                f"binary_sensor.climate_{room_name}_eco_mode",
                "on" if eco_active else "off",
                attributes={
                    "friendly_name": f"{room} Eco Mode",
                    "device_class": "power",
                    "icon": "mdi:leaf" if eco_active else "mdi:leaf-off",
                    "savings_estimate": self.controller.get_eco_savings_estimate(room)
                }
            )

        # Update system sensors
        season = self.controller.get_current_season()
        self.hass.set_state(
            "sensor.climate_season_detection",
            season,
            attributes={
                "friendly_name": "Weather-based Season",
                "icon": self._get_season_icon(season),
                "outdoor_temp": self.controller.get_outdoor_temp(),
                "detection_method": "weather_based",
                "last_updated": datetime.now().isoformat()
            }
        )

        power_source = self.controller.get_active_power_source()
        self.hass.set_state(
            "sensor.climate_power_source",
            power_source,
            attributes={
                "friendly_name": "Active Power Source",
                "icon": self._get_power_icon(power_source),
                "solar_available": self.controller.is_solar_available(),
                "grid_cost": self.controller.get_current_grid_cost(),
                "estimated_cost_per_hour": self.controller.get_estimated_hourly_cost()
            }
        )

    def _get_temp_status(self, room):
        """Get temperature status for a room"""
        current = self.controller.get_current_temp(room)
        target = self.controller.get_target_temp(room)
        diff = current - target

        if abs(diff) < 0.5:
            return "optimal"
        elif diff > 0:
            return "too_warm"
        else:
            return "too_cool"

    def _get_temp_status_text(self, temp_diff):
        """Get human readable temperature status"""
        if abs(temp_diff) < 0.5:
            return "Optimal"
        elif temp_diff > 0:
            return "Too Warm"
        else:
            return "Too Cool"

    def _get_action_icon(self, action):
        """Get icon for climate action"""
        icons = {
            "heating": "mdi:fire",
            "cooling": "mdi:snowflake",
            "off": "mdi:power-off",
            "idle": "mdi:pause",
            "eco": "mdi:leaf"
        }
        return icons.get(action, "mdi:hvac")

    def _get_season_icon(self, season):
        """Get icon for season"""
        icons = {
            "winter": "mdi:snowflake",
            "spring": "mdi:flower",
            "summer": "mdi:weather-sunny",
            "autumn": "mdi:leaf-maple",
            "fall": "mdi:leaf-maple"
        }
        return icons.get(season, "mdi:calendar")

    def _get_power_icon(self, power